In [1]:
import numpy as np
import pandas as pd
print(pd.__version__)
import glob, os, random, shutil, sys, time
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

#mac
util_path = "/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/util"

#constance
'''args = sys.argv[0:]
py_file = args[0]

code_location = os.path.dirname(os.path.abspath(py_file))
index_of_latest = code_location.index('latest')
util_path = os.path.join(code_location[:index_of_latest], "latest", "util")
'''

sys.path.insert(0, util_path)

# These two lines will automatically reload updated modules of mine
# (therefore, I don't need to restart kernel after editing importing modules)
#%load_ext autoreload
#%autoreload 2

1.3.5


In [2]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [3]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/2_bmd/2_analysis/Lindsay/best_model


In [22]:
complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Lindsay/12_DooNam_to_Lindsay/input/used_for_bmd/all/df_merged.csv'
#complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Lindsay/12_DooNam_to_Lindsay/input/used_for_bmd/each/Phthalate_mix.csv'

#df_test_dose_response = pd.read_csv(complete_file_path, header = 0)
df_test_dose_response = pd.read_csv(complete_file_path)
pd.set_option('display.max_columns', None)
display(df_test_dose_response.head())
display(df_test_dose_response.tail())
#display(np.unique(morpho_data.well))

,chemical.id,dose,total_num,num_affected
0,G14 Mix,0.0,36,4
1,G14 Mix,2.0,36,13
2,G14 Mix,4.0,36,22
3,G14 Mix,4.5,36,34
4,G14 Mix,5.0,36,32


,chemical.id,dose,total_num,num_affected
43,Benzyl butyl phthalate,3.5,36,5
44,Benzyl butyl phthalate,4.0,36,14
45,Benzyl butyl phthalate,4.5,36,21
46,Benzyl butyl phthalate,5.0,36,29
47,Benzyl butyl phthalate,8.0,36,36


In [32]:
# [goal] divide into csv files

full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(df_test_dose_response['chemical.id'])
else: # full_devel = "devel"
    #chemical_id_from_here = ['Benzophenone']
    chemical_id_from_here = ['Benzyl butyl phthalate']
    
start_time = time.time()

for chemical_id in chemical_id_from_here:
    print("chemical_id:" + str(chemical_id))
    
    # subset original dataframe for a user-specified chemical and end_point pair
    df_test_dose_response_per_chemical = df_test_dose_response.loc[df_test_dose_response['chemical.id'] == chemical_id,['chemical.id', 'dose', 'total_num', 'num_affected']]
 #   print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")
    
    # Fit dose response models
    df_test_dose_response_per_chemical = df_test_dose_response_per_chemical.sort_values(by=['dose'])    
#    print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")

    output_filename = str(chemical_id) + ".csv"
    df_test_dose_response_per_chemical.to_csv(output_filename, index=False) 
    
    df_test_dose_response_per_chemical = pd.read_csv(output_filename)
    
    df_test_dose_response_per_chemical = df_test_dose_response_per_chemical.sort_values(by=['dose'])    
    print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")
    
    model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response_per_chemical)
    #print (f"model_predictions:\n{model_predictions}")

    # Select best model
    selected_model_params = bmdest.select_model(model_predictions)
    #print (f"selected_model_params:{selected_model_params}")
    
    end_point = "Lindsay"
    bmd_feasibility_flag = 2

    # Check if unique model is found
    unique_model_flag = selected_model_params['no_unique_model_found_flag']
    #print (f"unique_model_flag:{unique_model_flag}")

    if(unique_model_flag == 0):
        # Generate report
        #print(df_test_dose_response.dose[-1:])

        ps.save_results_good_data_unique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
    else:
        bmd_analysis_flag = selected_model_params['model_select_flag']
        if(bmd_analysis_flag == 1):
            ps.save_results_poor_data_or_no_convergence(df_test_dose_response_per_chemical, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
        else:
            ps.save_results_good_data_nounique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took)) 


chemical_id:BMC10 Mix
df_test_dose_response_per_chemical:
  chemical.id   dose  total_num  num_affected
0   BMC10 Mix  0.000         36             2
1   BMC10 Mix  0.172         36             7
2   BMC10 Mix  0.343         36             3
3   BMC10 Mix  0.686         36             6
4   BMC10 Mix  1.370         36             9
5   BMC10 Mix  2.060         36            19
6   BMC10 Mix  2.750         36            28
7   BMC10 Mix  3.430         36            35
Model Convergence:True
chemical_id:Benzyl butyl phthalate
df_test_dose_response_per_chemical:
              chemical.id  dose  total_num  num_affected
0  Benzyl butyl phthalate  0.00         36             1
1  Benzyl butyl phthalate  2.25         36             0
2  Benzyl butyl phthalate  3.00         36             4
3  Benzyl butyl phthalate  3.50         36             5
4  Benzyl butyl phthalate  4.00         36            14
5  Benzyl butyl phthalate  4.50         36            21
6  Benzyl butyl phthalate  5.00    

In [ ]:
a=b

In [27]:
full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(df_test_dose_response['chemical.id'])
else: # full_devel = "devel"
    chemical_id_from_here = ['Phthalate mix']
    #chemical_id_from_here = ['G14 Mix']

        
print("\nchemical_id_from_here:" + str(chemical_id_from_here))
        
start_time = time.time()

#df_test_dose_response_copied = df_test_dose_response.copy()

for chemical_id in chemical_id_from_here:
    print("\nchemical_id:" + str(chemical_id))
    
 #   df_test_dose_response = df_test_dose_response_copied.copy()
    
    # subset original dataframe for a user-specified chemical
    df_test_dose_response_per_chemical = df_test_dose_response.loc[df_test_dose_response['chemical.id'] == chemical_id,['chemical.id', 'dose', 'total_num', 'num_affected']]

    
    #print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")
    

    # Fit dose response models
    df_test_dose_response_per_chemical = df_test_dose_response_per_chemical.sort_values(by=['dose'])    
    print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")


#    try:
    model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response_per_chemical)
    print (f"model_predictions:\n{model_predictions}")
#     except:
#         continue

    '''
    # Select best model
    selected_model_params = bmdest.select_model(model_predictions)
    print (f"selected_model_params:{selected_model_params}")
    
    end_point = "Lindsay"
    bmd_feasibility_flag = 2

    # Check if unique model is found
    unique_model_flag = selected_model_params['no_unique_model_found_flag']
    print (f"unique_model_flag:{unique_model_flag}")

    if(unique_model_flag == 0):
        # Generate report
        #print(df_test_dose_response.dose[-1:])

        ps.save_results_good_data_unique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
    else:
        bmd_analysis_flag = selected_model_params['model_select_flag']
        if(bmd_analysis_flag == 1):
            ps.save_results_poor_data_or_no_convergence(df_test_dose_response_per_chemical, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
        else:
            ps.save_results_good_data_nounique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
'''
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took)) 


chemical_id_from_here:['BMC10 Mix' 'Benzyl butyl phthalate' 'Diisobutyl phthalate' 'G14 Mix'
 'Phthalate mix' 'di-n-butyl phthalate']

chemical_id:BMC10 Mix
df_test_dose_response_per_chemical:
   chemical.id   dose  total_num  num_affected
8    BMC10 Mix  0.000         36             2
9    BMC10 Mix  0.172         36             7
10   BMC10 Mix  0.343         36             3
11   BMC10 Mix  0.686         36             6
12   BMC10 Mix  1.370         36             9
13   BMC10 Mix  2.060         36            19
14   BMC10 Mix  2.750         36            28
15   BMC10 Mix  3.430         36            35
Model Convergence:True


KeyError: 0